In [ ]:
from langchain_community.document_loaders import YoutubeLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain_core.prompts import (SystemMessagePromptTemplate,
                                    HumanMessagePromptTemplate,
                                    ChatPromptTemplate)
from langchain_core.output_parsers import StrOutputParser

In [ ]:
import yaml
import logging
from datetime import datetime

# YAML config
try:
    with open(r".\config.yaml", "r") as f:
        config = yaml.safe_load(f)
except Exception as e:
    raise

# Logger
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(name)s - %(funcName)s - %(message)s",
    filename=config["log_dir"] +
    f"{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.log",
    filemode="w"
)
logger = logging.getLogger(__name__)

logger.info("Config file and logger setup completed.")

In [ ]:
def extract_and_summarize_youtube_content(video_url):
    """
    Extracts text from a YouTube video and generates a short summary using Ollama.

    Args:
        video_url (str): The URL of the YouTube video.

    Returns:
        str: A short summary of the video content.  Returns None if an error occurs.
    """
    try:
        # Load the YouTube document
        loader = YoutubeLoader.from_youtube_url(
            video_url, add_video_info=False)
        docs = loader.load()

        if not docs:
            logger.warning(f"No documents loaded from {video_url}")
            return None

        doc = docs[0]  # Assuming only one document is loaded

        # Split the text into chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000, chunk_overlap=100)
        chunks = text_splitter.split_text(doc.page_content)

        system = SystemMessagePromptTemplate.from_template(
            """You are a helpful AI assistant specializing in concise summarization of text content.""")
        prompt = """Create a short summarization of the provided context!
            ### Context:
            {context}

            ### Answer:"""

        llm = ChatOllama(model=config["model"], base_url="url")
        prompt = HumanMessagePromptTemplate.from_template(prompt)
        template = ChatPromptTemplate(messages=[system, prompt])
        qna_chain = template | llm | StrOutputParser()

        sums = []
        for chunk in chunks:
            sum = qna_chain.invoke({"context": chunk})
            sums.append(sum)

        # Combine summaries and final prompt
        sums_join = ". ".join([sum.split("\n\n")[-1] for sum in sums])
        result = qna_chain.invoke({"context": sums_join})
        return result.strip()

    except Exception as e:
        logger.error(f"An error occurred: {e}")
        return None

In [ ]:
print(extract_and_summarize_youtube_content("url"))